In [5]:
import numpy as np

# AES Rcon
RCON = [0x01,0x02,0x04,0x08,0x10,0x20,0x40,0x80,0x1B,0x36]

# AES S-box
SBOX = np.array([
  0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76,
  0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0,
  0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15,
  0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75,
  0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84,
  0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf,
  0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8,
  0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2,
  0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73,
  0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb,
  0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79,
  0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08,
  0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a,
  0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e,
  0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf,
  0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16
], dtype=np.uint8)

def inv_key_schedule(last_round_key_hex):
    """Invert AES-128 key schedule: take round10 key -> return master key"""
    w = [0]*44  # 44 words of 4 bytes
    key_bytes = bytes.fromhex(last_round_key_hex)
    for i in range(4):
        w[40+i] = [key_bytes[4*i+j] for j in range(4)]

    # invert key expansion
    for i in range(39, -1, -1):
        if i % 4 == 0:
            temp = w[i+4][:]
            # rotate
            temp = temp[1:] + temp[:1]
            # sub
            temp = [SBOX[b] for b in temp]
            # Rcon
            temp[0] ^= RCON[i//4]
            w[i] = [ (w[i+4][j] ^ temp[j]) & 0xff for j in range(4) ]
        else:
            w[i] = [ (w[i+4][j] ^ w[i+3][j]) & 0xff for j in range(4) ]

    # master key is w[0..3]
    master_key = sum(w[0:4], [])
    return bytes(master_key).hex().upper()


# Example usage:
k10 = "FF539081B852063509E71D41DA49B5D2"  # <-- replace with your recovered round10 key
master = inv_key_schedule(k10)
print("Recovered Master Key:", master)


Recovered Master Key: 076EAC45E90F1C6C3EED0FFD41698543


In [6]:
import numpy as np
from Crypto.Cipher import AES

# ======================
# AES Constants
# ======================
RCON = [0x01,0x02,0x04,0x08,0x10,0x20,0x40,0x80,0x1B,0x36]

SBOX = np.array([
  0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76,
  0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0,
  0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15,
  0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75,
  0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84,
  0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf,
  0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8,
  0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2,
  0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73,
  0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb,
  0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79,
  0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08,
  0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a,
  0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e,
  0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf,
  0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16
], dtype=np.uint8)

# ======================
# Invert Key Schedule
# ======================
def inv_key_schedule(last_round_key_hex):
    """Invert AES-128 key schedule: K10 -> K0"""
    w = [0]*44  # 44 words of 4 bytes
    key_bytes = bytes.fromhex(last_round_key_hex)
    for i in range(4):
        w[40+i] = [key_bytes[4*i+j] for j in range(4)]

    # invert expansion
    for i in range(39, -1, -1):
        if i % 4 == 0:
            temp = w[i+4][:]
            temp = temp[1:] + temp[:1]          # rotate
            temp = [SBOX[b] for b in temp]      # sub
            temp[0] ^= RCON[i//4]               # add Rcon
            w[i] = [(w[i+4][j] ^ temp[j]) & 0xff for j in range(4)]
        else:
            w[i] = [(w[i+4][j] ^ w[i+3][j]) & 0xff for j in range(4)]

    master_key = sum(w[0:4], [])
    return bytes(master_key)

# ======================
# Example: Recover + Verify
# ======================
# Replace with your CPA result (K10)
k10 = "FF539081B852063509E71D41DA49B5D2"  

# Invert to get master key
master_key = inv_key_schedule(k10)
print("Recovered Master Key:", master_key.hex().upper())

# ----------------------
# Verification step
# ----------------------
PLAINTEXT_FILE  = "r_pt.txt"
CIPHERTEXT_FILE = "r_ct.txt"

plaintexts  = [bytes.fromhex(l.strip()) for l in open(PLAINTEXT_FILE)]
ciphertexts = [bytes.fromhex(l.strip()) for l in open(CIPHERTEXT_FILE)]

cipher = AES.new(master_key, AES.MODE_ECB)
ok = True
for pt, ct in zip(plaintexts, ciphertexts):
    if cipher.encrypt(pt) != ct:
        ok = False
        break

if ok:
    print("Verification: ✔️ PASSED (correct master key!)")
else:
    print("Verification: ❌ FAILED (something off)")


Recovered Master Key: 076EAC45E90F1C6C3EED0FFD41698543
Verification: ❌ FAILED (something off)
